# DreamBooth SDXL 1.0 – full‑model fine‑tune with a custom style token `<kmk>`
This Colab notebook sets up everything (dependencies, data folders, script, training and inference) to train a **full Stable Diffusion XL model** using DreamBooth. It is optimised for a single **NVIDIA A100** GPU (40‑80 GB).

In [ ]:

# ⚙️ Runtime check
import torch, os, platform
print("🔋 CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("🖥️  GPU:", torch.cuda.get_device_name(0))
assert torch.cuda.is_available(), "❌  Please switch Colab to: Runtime ▸ Change runtime type ▸ GPU (A100 preferred)"


In [ ]:

# 🛠 Install / upgrade dependencies
!pip -q install --upgrade diffusers[torch] transformers accelerate bitsandbytes xformers --progress-bar off


In [ ]:

# 🔑 Hugging Face token (skip if model is public)
from getpass import getpass
HF_TOKEN = ""  # <-- paste your HF token if needed
if HF_TOKEN:
    from huggingface_hub import login
    login(HF_TOKEN, add_to_git_credential=True)


In [ ]:

# 📂 Mount Google Drive (optional, keeps checkpoints)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
OUTPUT_DIR = "/content/drive/MyDrive/sdxl-dreambooth-kmk"  # change if desired


In [ ]:

# 🖼 Upload instance images
INSTANCE_DIR = "/content/instance_imgs"
!mkdir -p $INSTANCE_DIR
print(f"✅  Upload 5‑20 images into {INSTANCE_DIR} via the left sidebar (folder icon).")


In [ ]:
# 📝 Training script path
SCRIPT_PATH = 'train_dreambooth_sdxl_full.py'
print(f'Using training script {SCRIPT_PATH}')


In [ ]:

# ⚡️ Generate accelerate default config
!accelerate config default


In [ ]:
# 🚀 Start DreamBooth training
!accelerate launch train_dreambooth_sdxl_full.py \
  --pretrained_model_name_or_path stabilityai/stable-diffusion-xl-base-1.0 \
  --instance_data_dir $INSTANCE_DIR \
  --instance_prompt 'a <kmk> style portrait of a corgi' \
  --resolution 1024 \
  --train_batch_size 1 \
  --learning_rate 1e-5 \
  --gradient_checkpointing \
  --mixed_precision bf16 \
  --max_train_steps 1200 \
  --output_dir $OUTPUT_DIR


In [ ]:

# ✅ Inference after training
from diffusers import StableDiffusionXLPipeline
import torch, matplotlib.pyplot as plt

pipe = StableDiffusionXLPipeline.from_pretrained(OUTPUT_DIR, torch_dtype=torch.bfloat16).to("cuda")
prompt = "a <kmk> style portrait of a Shiba Inu, sharp focus, 8k"
with torch.cuda.amp.autocast("cuda", dtype=torch.bfloat16):
    img = pipe(prompt, num_inference_steps=25).images[0]
plt.imshow(img); plt.axis("off")
